In [1]:
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
from utils import load_imagenet

In [2]:
(x_train, y_train), (x_val, y_val) = load_imagenet(8)

: 

: 

In [ ]:
x_train.shape, x_train.dtype